<h1 style="text-align:center">Shell.AI EV Problem (LP part) </h1>

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pulp import *
import numpy as np
import pandas as pd

In [ ]:
Dist = np.random.randn(4096,100)

In [ ]:
# so we have here is existing infrastructure and we need to model the different data points(supply) 
#that goes with the demand points
supply_points_existing = pd.read_csv("./exisiting_EV_infrastructure_2018.csv")
supply_points_existing

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5,3
1,1,37.336451,58.119225,27,4,7
2,2,46.709232,57.525650,31,6,14
3,3,30.528626,55.379835,26,5,5
4,4,51.521781,35.116755,32,11,6
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4
96,96,30.318396,33.388335,32,5,10
97,97,36.218839,22.235766,32,4,14
98,98,42.936915,38.122442,28,7,5


In [ ]:
demand=pd.read_csv("./Demand_predicted_nbeats.csv")
demand.head()

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,16.510711,22.027955
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,18.846287,18.269391
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,16.042624,21.742603
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,19.985781,24.266968
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,18.730846,27.517960


In [ ]:
demand_2018 = demand["2018"].to_numpy()
np.sum(demand_2018) # 2018 EXISTING DEMAND

361529.636596884

In [ ]:
demand_2019 = demand["2019"]

In [ ]:
np.sum(demand_2019)

493462.16857066

In [ ]:
#distance matrix between demand points and supply points
xs=np.array(supply_points_existing["x_coordinate"])
ys=np.array(supply_points_existing["y_coordinate"])
xd=np.array(demand["x_coordinate"])
yd=np.array(demand["y_coordinate"])
dist=np.zeros((100,4096))

for i in range(100):
    for j in range(4096):
        dist[i][j]=(((xs[i]-xd[j])**2)+((ys[i]-yd[j])**2))**0.5

dist=dist.T

In [ ]:
SCS = supply_points_existing["existing_num_SCS"].to_numpy()
FCS = supply_points_existing["existing_num_FCS"].to_numpy()
parking_slots = supply_points_existing["total_parking_slots"].to_numpy()

In [ ]:
np.sum(200*SCS + 400*FCS) # EXISTING SUPPLY

361600

2019

In [ ]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The EV problem", LpMinimize)

# setting up the required LP variables in the problems
supply_vars_SCS = LpVariable.dict("SCS", (range(100)), 0,None, cat=LpInteger) # minimum value is zero(constraint - 1)
supply_vars_FCS = LpVariable.dict("FCS", (range(100)), 0,None, cat=LpInteger) # minimum value is zero ( constraint - 1)
demand_supply_matrix = LpVariable.dict("Demand", (range(4096),range(100)), 0,None, cat=LpContinuous) 
# demand is varying continuously as given in the data (constraint 1+2)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
np.size(dist)

409600

In [ ]:
demand_supply_matrix[0,1].value()

In [ ]:
# The objective function is added to 'prob' first
prob += (
    lpSum([
          [
            (dist[i][j]*(demand_supply_matrix[i,j])) 
            for i in range(4096)] for j in range(100)
         ]
    )+
        lpSum([
          
            ((supply_vars_SCS[j] + 1.5*supply_vars_FCS[j])) 
             for j in range(100)
         ]
        ), 
    "Total Cost Function",
)

In [ ]:
# 3) Sum of FCS and SCS at jth point must be less than the total parking slots at the point
for j in range(100):
  prob += (
    lpSum(supply_vars_SCS[j] + supply_vars_FCS[j]) <= parking_slots[j],
    f"Parking_max {j}",
  )

In [ ]:
# Now we are defining the constraints to the problem here that are needed to be solved 
# 4) the infrastructure needs to be built on the existing infrastructure
for j in range(100):
  prob += (
    supply_vars_SCS[j] >= SCS[j],
    f"SCS_min_{j}",
  ) 
for j in range(100):
  prob += (
    supply_vars_FCS[j] >= FCS[j],
    f"FCS_min_{j}",
  )

In [ ]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
for j in range(100):
  prob += (
   lpSum(200*supply_vars_SCS[j] + 400*supply_vars_FCS[j]) - lpSum([demand_supply_matrix[i,j] for i in range(4096)])>= 0.001,
   f"demand_{j}",
  ) 

In [ ]:
# 6) Demand satisfed by each ith point should equal to the demand at the time
for i in range(4096):
  prob += (
    lpSum([demand_supply_matrix[i,j] for j in range(100)]) == demand_2019[i],
    f"demand_match_{i}",
  ) 

In [ ]:
prob.solve()

-1

In [ ]:
demand_supply_matrix_2019 = np.zeros((4096,100))
for i in range(4096):
  for j in range(100):
    demand_supply_matrix_2019[i][j] = demand_supply_matrix[i,j].value()

In [ ]:
np.sum(demand_supply_matrix_2019)

492499.6876468

In [ ]:
SCS_2019 = np.zeros(100)
FCS_2019 = np.zeros(100)
for i in range(100):
    SCS_2019[i] = supply_vars_SCS[i].value()
for i in range(100):
    FCS_2019[i] = supply_vars_FCS[i].value()

In [ ]:
np.sum(200*SCS_2019 + 400*FCS_2019)

560544.833844

In [ ]:
# Value of cost function I got here
cost_2019 = 0
for i in range(4096):
  for j in range(100):
      cost_2019 += (dist[i][j]*(demand_supply_matrix_2019[i][j])) 
for j in range(100):
  cost_2019 += 600*(SCS_2019[j] + 1.5*FCS_2019[j])


In [ ]:
cost_2019

3025144.6519013834

In [ ]:
score_2018 = max(10, 100 - ((90*cost_2019)/15000000))
score_2018

81.8491320885917

In [ ]:
for i in range(4096):
  for j in range(100):
    if demand_supply_matrix_2019[i][j] < 0:
      demand_supply_matrix_2019[i][j] = 0

In [ ]:
# 4) the infrastructure needs to be built on the existing infrastructure
constraint_4 = True
for i in range(100):
  if (SCS_2019[i] < SCS[i]):
    SCS_2019[i] = SCS[i]
  elif (FCS_2019[i] < FCS[i]):
    FCS_2019[i] = FCS[i]
    

Constraint-5 correction

In [ ]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
constraint_5 = True
constraint_5_array = []
for j in range(100):
  if np.sum(demand_supply_matrix_2019[i][j] for i in range(4096)) > (200*SCS_2019[j]+400*FCS_2019[j]):
    constraint_5 = False
    print(j)
    constraint_5_array.append(j)

0
10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  """


In [ ]:
len(demand_supply_matrix_2019[4095])

100

In [ ]:
j = 64
np.sum(demand_supply_matrix_2019[i][j] for i in range(4096))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


7399.999012000001

In [ ]:
(200*SCS_2019[j]+400*FCS_2019[j])

7400.0

In [ ]:
for j in constraint_5_array:
  for i in range(4096):
    if (demand_supply_matrix_2019[i][j] > 0):
      if demand_supply_matrix_2019[i][j] > np.sum(demand_supply_matrix_2019[i][j] for i in range(4096)) -(200*SCS_2019[j]+400*FCS_2019[j]) + 1:
        demand_supply_matrix_2019[i][j] -= np.sum(demand_supply_matrix_2019[i][j] for i in range(4096)) -(200*SCS_2019[j]+400*FCS_2019[j])
        break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  """


In [ ]:
constraint_5

False

In [ ]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
constraint_5 = True
constraint_5_array = []
for j in range(100):
  if np.sum(demand_supply_matrix_2019[i][j] for i in range(4096)) > (200*SCS_2019[j]+400*FCS_2019[j]):
    constraint_5 = False
    print(j)
    constraint_5_array.append(j)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  """


Checking constraints

In [ ]:
# 1st and 2nd constraints must be true as they are initialized using bounded conditions(0-Infinity)

In [ ]:
constraint_1 = True
for i in SCS_2019:
  if i<0:
    constraint_1 = False
    print(i)
for i in FCS_2019:
  if i<0:
    constraint_1 = False
    print(i)

In [ ]:
constraint_2 = True
for i in range(4096):
  for j in range(100):
    if demand_supply_matrix_2019[i][j] < 0:
      constraint_2 = False
      print(i,j)

In [ ]:
# checking constraints
# 3) Sum of FCS and SCS at jth point must be less than the total parking slots at the point
constraint_3 = True
for i in range(100):
  if (SCS_2019[i] + FCS_2019[i]) > parking_slots[i]:
    constraint_3 = False
    print(i)

In [ ]:
# 4) the infrastructure needs to be built on the existing infrastructure
constraint_4 = True
for i in range(100):
  if (SCS_2019[i] < SCS[i]) or (FCS_2019[i] < FCS[i]):
    constraint_4 = False
    print(i)

In [ ]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
constraint_5 = True
constraint_5_array = []
for j in range(100):
  if np.sum(demand_supply_matrix_2019[i][j] for i in range(4096)) > (200*SCS_2019[j]+400*FCS_2019[j]):
    constraint_5 = False

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  """


In [ ]:
# 6) Demand satisfed by each ith point should equal to the demand at the time
constraint_6 = True
for i in range(4096):
  if (np.sum(demand_supply_matrix_2019[i][j] for j in range(100)) != demand_2019[i]):
    constraint_6 = False
    #print(i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  after removing the cwd from sys.path.


In [ ]:
print("constraint 1:",constraint_1)
print("constraint 2:",constraint_2)
print("constraint 3:",constraint_3)
print("constraint 4:",constraint_4)
print("constraint 5:",constraint_5)
print("constraint 6:",constraint_6)

constraint 1: True
constraint 2: True
constraint 3: True
constraint 4: True
constraint 5: True
constraint 6: False


2020

In [ ]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The EV problem-2", LpMinimize)

# setting up the required LP variables in the problems
supply_vars_SCS = LpVariable.dict("SCS", (range(100)), 0,None, cat=LpInteger) # minimum value is zero(constraint - 1)
supply_vars_FCS = LpVariable.dict("FCS", (range(100)), 0,None, cat=LpInteger) # minimum value is zero ( constraint - 1)
demand_supply_matrix = LpVariable.dict("Demand", (range(4096),range(100)), 0,None, cat=LpContinuous) 
# demand is varying continuously as given in the data (constraint 1+2)

In [ ]:
# The objective function is added to 'prob' first
prob += (
    lpSum([
          [
            (dist[i][j]*(demand_supply_matrix[i,j])) 
            for i in range(4096)] for j in range(100)
         ]
    )+
        lpSum([
          
            ((supply_vars_SCS[j] + 1.5*supply_vars_FCS[j])) 
             for j in range(100)
         ]
        ), 
    "Total Cost Function",
)

In [ ]:
# 3) Sum of FCS and SCS at jth point must be less than the total parking slots at the point
for j in range(100):
  prob += (
    lpSum(supply_vars_SCS[j] + supply_vars_FCS[j]) <= parking_slots[j],
    f"Parking_max {j}",
  )

In [ ]:
# Now we are defining the constraints to the problem here that are needed to be solved 
# 4) the infrastructure needs to be built on the existing infrastructure
for j in range(100):
  prob += (
    supply_vars_SCS[j] >= SCS_2019[j],
    f"SCS_min_{j}",
  ) 
for j in range(100):
  prob += (
    supply_vars_FCS[j] >= FCS_2019[j],
    f"FCS_min_{j}",
  )

In [ ]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
for j in range(100):
  prob += (
   lpSum([demand_supply_matrix[i,j] for i in range(4096)]) - 
   lpSum(200*supply_vars_SCS[j] + 400*supply_vars_FCS[j]) <= 0.001,
   f"demand_{j}",
  ) 

In [ ]:
demand_2020 = demand["2020"]

In [ ]:
# 6) Demand satisfed by each ith point should equal to the demand at the time
for i in range(4096):
  prob += (
    lpSum([demand_supply_matrix[i,j] for j in range(100)]) == demand_2020[i],
    f"demand_match_{i}",
  ) 

In [ ]:
prob.solve()

In [ ]:
demand_supply_matrix_2020 = np.zeros((4096,100))
for i in range(4096):
  for j in range(100):
    demand_supply_matrix_2020[i][j] = demand_supply_matrix[i,j].value()

In [ ]:
np.sum(demand_supply_matrix_2020)

578797.8248825

In [ ]:
SCS_2020 = np.zeros(100)
FCS_2020 = np.zeros(100)
for i in range(100):
    SCS_2020[i] = supply_vars_SCS[i].value()
for i in range(100):
    FCS_2020[i] = supply_vars_FCS[i].value()

In [ ]:
np.sum(200*SCS_2020 + 400*FCS_2020)

627126.1267200001

In [ ]:
# Value of cost function I got here
cost_2020 = 0
for i in range(4096):
  for j in range(100):
      cost_2020 += (dist[i][j]*(demand_supply_matrix_2020[i][j])) 
for j in range(100):
  cost_2020 += 600*(SCS_2020[j] + 1.5*FCS_2020[j])
cost_2020

3650000.5822192584

In [ ]:
score_2020 = max(10, 100 - ((90*cost_2020)/15000000))
score_2020

78.09999650668445

In [ ]:
for i in range(4096):
  for j in range(100):
    if demand_supply_matrix_2020[i][j] < 0:
      demand_supply_matrix_2020[i][j] = 0

In [ ]:
constraint_3 = True
for i in range(100):
  if (SCS_2020[i] + FCS_2020[i]) > parking_slots[i]:
    

Correcting constraint 5:

In [ ]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
constraint_5 = True
constraint_5_array = []
for j in range(100):
  if np.sum(demand_supply_matrix_2020[i][j] for i in range(4096)) > (200*SCS_2020[j]+400*FCS_2020[j]):
    constraint_5 = False
    print(j)
    constraint_5_array.append(j)

In [ ]:
len(demand_supply_matrix_2020[4095])

In [ ]:
for j in constraint_5_array:
  for i in range(4096):
    if (demand_supply_matrix_2020[i][j] > 0):
      demand_supply_matrix_2020[i][j] -= 0.5
      break

In [ ]:
j = 34
np.sum(demand_supply_matrix_2020[i][j] for i in range(4096))

In [ ]:
(200*SCS_2020[j]+400*FCS_2020[j])

Checking Constraints

In [ ]:
# 1st and 2nd constraints must be true as they are initialized using bounded conditions(0-Infinity)
constraint_1 = True
for i in SCS_2020:
  if i<0:
    constraint_1 = False
    print(i)
for i in FCS_2020:
  if i<0:
    constraint_1 = False
    print(i)
constraint_2 = True
for i in range(4096):
  for j in range(100):
    if demand_supply_matrix_2020[i][j] < 0:
      constraint_2 = False
      print(i,j)

In [ ]:
# checking constraints
# 3) Sum of FCS and SCS at jth point must be less than the total parking slots at the point
constraint_3 = True
for i in range(100):
  if (SCS_2020[i] + FCS_2020[i]) > parking_slots[i]:
    constraint_3 = False
    print(i)

In [ ]:
# 4) the infrastructure needs to be built on the existing infrastructure
constraint_4 = True
for i in range(100):
  if (SCS_2020[i] < SCS_2019[i]) or (FCS_2020[i] < FCS_2019[i]):
    constraint_4 = False
    print(i)

In [ ]:
# 5) Demand satisfed by each jth supply point should be less than or equal to the maximum supply available
constraint_5 = True
constraint_5_array = []
for j in range(100):
  if np.sum(demand_supply_matrix_2020[i][j] for i in range(4096)) > (200*SCS_2020[j]+400*FCS_2020[j]):
    constraint_5 = False
    print(j)
    constraint_5_array.append(j)

In [ ]:
# 6) Demand satisfed by each ith point should equal to the demand at the time
constraint_6 = True
for i in range(4096):
  if np.sum(demand_supply_matrix_2020[i][j] for j in range(100)) != demand_2020[i]:
    constraint_6 = False
    #print(i)

In [280]:
print("constraint 1:",constraint_1)
print("constraint 2:",constraint_2)
print("constraint 3:",constraint_3)
print("constraint 4:",constraint_4)
print("constraint 5:",constraint_5)
print("constraint 6:",constraint_6)

constraint 1: True
constraint 2: False
constraint 3: False
constraint 4: True
constraint 5: True
constraint 6: False


In [ ]:
demand

In [ ]:
'''
fileh = open("submit-19.csv", "r+")
fileh.write("year,data_type,demand_point_index,supply_point_index,value\n")
for i in range(100):
    fileh.write(f"2019,SCS,,{i},{int(SCS_2019[i])}\n")
for i in range(100):
    fileh.write(f"2019,FCS,,{i},{int(FCS_2019[i])}\n")
for i in range(4096):
    for j in range(100):
        fileh.write(f"2019,DS,{i},{j},{demand_supply_matrix_2019[i][j]}\n")
for i in range(100):
    fileh.write(f"2020,SCS,,{i},{int(SCS_2020[i])}\n")
for i in range(100):
    fileh.write(f"2020,FCS,,{i},{int(FCS_2020[i])}\n") 
for i in range(4096):
    for j in range(100):
        fileh.write(f"2020,DS,{i},{j},{demand_supply_matrix_2020[i][j]}\n")
fileh.write("year,data_type,demand_point_index,supply_point_index,value\n")
fileh.close()
'''